In [1]:
import numpy as np
import pandas as pd

## Importing data
This reads the spreadsheet stored at `data/root.xlsx`. The first step is to replace the column names to make it more amenable to programmatic referencing.

In [2]:
PATH = "data/root.xlsx"
SHEET = "Data"

# read
df = pd.read_excel(PATH, SHEET)
df.columns = [
    "nhs_num",
    "lab_num",
    "age",
    "gender",
    "location",
    "location_other",
    "biopsy_type",
    "size",
    "mdm2",
    "karyo_code",
    "diagnosis",
    "diagnosis_other",
    "follow_up",
    "mortality",
    "cancer_mortality",
    "cancer_mortality_other",
    "comment",
    "query",
    "misc"
]

df

/opt/miniconda3/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,nhs_num,lab_num,age,gender,location,location_other,biopsy_type,size,mdm2,karyo_code,diagnosis,diagnosis_other,follow_up,mortality,cancer_mortality,cancer_mortality_other,comment,query,misc
0,7079315611,LH24-19172,NaN,NaN,NaN,NaN,NaN,NaN,Non-amplified,MDM2 normal,NaN,NaN,NaN,alive,NaN,NaN,CF,NaN,NaN
1,4509208588,LH24-20991,74,Female,O - Other (Free- text),Breast,NaN,20,Non-amplified,MDM2 normal,Lipoma,NaN,NaN,alive,NaN,NaN,CF,NaN,NaN
2,6342778353,LH24-18651,45,Female,HN - Head and Neck,Right cheek parotid gland,NaN,71,Non-amplified,MDM2 normal,Lipoma,NaN,NaN,alive,NaN,NaN,CF,Should parotid gland be in other?,NaN
3,6415674862,LH24-20355,23,Female,LLS - Lower limbs (Superficial),NaN,core,NaN,Non-amplified,MDM2 normal,Other (free text),Myxoid liposarcoma,NaN,alive,NaN,NaN,CF,Unsure whether diagnosis fits in option 'Pleom...,NaN
4,4520257115,LH24-19486,73,Female,ULS - Upper limbs (Superficial),NaN,NaN,15,Non-amplified,MDM2 normal,Lipoma,NaN,NaN,alive,NaN,NaN,CF,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2701,6049456305,BH09-3046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2702,4064416066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2703,4343743969,LH09-10296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2704,4382331137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MDM2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Cleaning data
Defining some common column cleaning utilities. Excel stores some blank cells with `\xa0`. These must be converted into plain empty strings - something that will be repeated for other fields.

*TODO - unspecified blank*

In [3]:
def std_clean(sheet, col, is_str=True):
    if is_str:
        sheet[col] = sheet[col].str.strip().str.lower()
    sheet.loc[sheet[col] == "\xa0", col] = np.nan
    sheet.loc[sheet[col] == "", col] = np.nan

### Age

In [4]:
std_clean(df, "age", is_str=False)

df["age"].unique()

array([nan, 74, 45, 23, 73, 79, 68, 70, 57, 38, 56, 28, 75, 41, 82, 69,
       52, 12, 85, 55, 65, 64, 89, 42, 35, 80, 54, 43, 63, 51, 16, 61, 46,
       91, 86, 47, 33, 67, 62, 66, 77, 76, 39, 44, 81, 84, 60, 50, 83, 87,
       48, 32, 19, 26, 31, 49, 40, 53, 58, 22, 59, 34, 72, 25, 71, 15, 27,
       18, 29, 78, 36, 37, 88, 13, 98, 100, 17, 30, 20, 93, 90, 92, 94, 6,
       1, 4, 8, 24, 21, 14], dtype=object)

### Gender
Standardising gender into "m", "f" and NaN for missing or "unspecified" data.

In [5]:
std_clean(df, "gender")
df.loc[df["gender"] == "unspecified", "gender"] = np.nan
df.loc[df["gender"] == "male", "gender"] = "m"
df.loc[df["gender"] == "female", "gender"] = "f"

df["gender"].unique()

array([nan, 'f', 'm'], dtype=object)

### Location
The spreadsheet encodes this in the format `{code} - {description}`. Since the description is the same for every code, we are filtering only the code.

In [6]:
std_clean(df, "location")
# NaN modification
idxs = ~df["location"].isna()
# df.loc[idxs, "location"] = df.loc[idxs, "location"] \
#     .apply(lambda x: x.split("-")[0].strip())

df["location"].unique()

array([nan, 'o - other (free- text)', 'hn - head and neck',
       'lls - lower limbs (superficial)',
       'uls - upper limbs (superficial)', 'uld - upper limbs (deep)',
       't- trunk', 'lld - lower limbs (deep)', 'lls - lower limbs (deep)',
       't - trunk'], dtype=object)

### Location (other)
*TODO*
row 1899 has lld in wrong column

In [43]:
std_clean(df, "location_other")

df["location_other"].unique()

array([nan, 'breast', 'right cheek parotid gland', 'spermatic cord',
       'left testicle', 'right kidney', 'groin', 'right parotid',
       'left hemiscrotum', 'left groin', 'right groin', 'tongue',
       'thrombosed varix', 'left pararenal mass',
       'shoulder blade soft tissue', 'left kidney', 'left paraspinal',
       'right 6th rib', 'presacral', 'r posterior shoulder',
       'left adrenal gland', 'right posterior arm, shoulder plus back',
       'axilla', 'retroperitoneal', 'thymus',
       'inguinal fat (left scrotal hernia)', 'vaginal wall polyp',
       'axillary core biopsy', 'small bowel mesenteric mass',
       'mesenteric mass', 'hilar mass', 'anterior mediastinal mass',
       'plueral debris', 'left scrotal lesion', 'inguinal orchidectomy',
       'right retroperitoneum (iliopsoas)', 'breast l',
       'right inguinal orchidectomy', 'mediastinal mass',
       'retroperitoneum',
       'right inguinal orchidectomy plus scrotal lesion',
       'retroperitoneal mass',

### Size
*TODO - split into size and sample method, comma sep sizes*

In [44]:
std_clean(df, "size", is_str=False)

df["size"].unique()

array([nan, 20, 71, 15, 14, 159, 13, 110, 75, 8, 12, 18, 70, 85, 9, 17,
       55, 150, 195, 26, 116, 28, 52, 16, 11, 160, 60, 200, 135, 165, 10,
       300, 19, 81, 80, 50, 100, 6, 90, 240, 95, 25, 21, 40, 330, 180,
       120, 190, 250, 68, 22, 23, 145, 65, 45, 105, 305, 7, 170, 34, 24,
       4, 5, 35, 3, 42, 130, 62, 63, 36, 30, 32, 59, 290, 111, 185, 103,
       39, 140, 216, 166, 115, 27, 125, 84, 78, 38, 136, 220, 205, 221,
       89, 102, 162, 104, 72, 43, 101, 94, 47, 215, 41, 265, 109, 107,
       175, 280, 1, 88, 97, 93, 33, 310, 37, 73, 92, 183, 350, 2.5, 2,
       124, 58, 76, 108, 225, 230, 113, 155, 245, 74, 177, 82, 51, 86,
       210, 67, 98, 96, 49, 117, 64, 114, 168, 126, 380, 54, 390, 77, 260,
       199, 29, 360, 31, 151, 3.5, 112, 83, 400, 500, 57, 1110, 48, 208],
      dtype=object)

### MDM2
Now transformed to represent MDM2 amplification as a boolean.

In [45]:
std_clean(df, "mdm2")
df.loc[df["mdm2"] == "non-amplified", "mdm2"] = False
df.loc[df["mdm2"] == "amplified", "mdm2"] = True

df["mdm2"].unique()

array([False, True, nan], dtype=object)

### Karyo Code
*TODO - purpose?*

In [46]:
std_clean(df, "karyo_code")

df["karyo_code"].unique()

array(['mdm2 normal', 'mdm2 amp', nan, 'amplified', 'fish failed',
       'fish normal', 'fish ab', 'abnormal', 'dleu deleted'], dtype=object)

### Diagnosis
*TODO*

In [47]:
std_clean(df, "diagnosis")

df["diagnosis"].unique()

array([nan, 'lipoma', 'other (free text)',
       'well differentiated liposarcoma', 'liposarcoma',
       'atypical lipomatous tumour', 'dedifferentiated liposarcoma',
       'pleomorphic liposarcoma', 'sclerosing mesenteritis',
       'pleomorphic myxoid liposarcoma', 'low-grade leiomyosarcoma',
       'fibrolipoma', 'poorly differentiated sarcoma',
       'undifferentiated liposarcoma'], dtype=object)

### Diagnosis (Other)
*TODO*

In [48]:
std_clean(df, "diagnosis_other")

df["diagnosis_other"].unique()

array([nan, 'myxoid liposarcoma',
       'fibromycoid tumour of uncertain \nmalignant potential',
       'metastatic nonseminomatous germ cell \ntumour, teratoma with focal somatic malignancy (sarcoma, showing areas of rhabdomyloblastic differentiation)',
       'recurrent', 'lipoblastoma', 'scarring process',
       'atypical spindle cell lipomatous tumour', 'myolipoma',
       'igg4 related disease?', 'fibrous dysplasia',
       'atypical lipomatous tumour/well \ndifferentiated liposarcoma, grade 1',
       'atypical lipomatous tumour/well \ndifferentiated liposarcoma, grade 2',
       'fat necrosis',
       'borderline fibroblastic tumour with\nsome features concerning for malignancy',
       'spindle cell lipoma', 'solitary fibrous tumour',
       'spindle cell lipoma, with a minimal \nadipocytic component',
       'spindle cell soft tissue tumour of uncertain \nmalignant potential',
       'spindle cell', 'likely metastatic carcinoma',
       'localised type tenosynovial giant cel

### Follow Up
*TODO*

In [49]:
std_clean(df, "follow_up")

df["follow_up"].unique()

array([nan, 'death 2025.02.11', 'death 2024.07.15', 'death 2024.02.03',
       'death 2025.01.11', 'death 2024.03.29',
       'tumour right thigh mixed type im angioma',
       'tumour right arm myxofibrosarcoma',
       'pleomorphic atypical lipomatous tumour',
       'dedifferentiated liposarcoma fnclcc grade 3',
       'a) retroperitoneal resection: well-differentiated liposarcoma b) colon: within normal limits',
       'myxoid tumour with no features of malignancy',
       'recurrent liposarcoma posterior left thigh',
       'cellular angiofibroma', 'death 2023.12.30',
       'lipoma left shoulder', 'spindle cell lipoma',
       'right anterior shoulder haemangioma',
       'excision, mass back: grade 3 spindle cell sarcoma',
       'well differentiated liposarcoma with focal low-grade dedifferentiation',
       'high grade myxoid sarcoma', 'intramuscular lipomatous tumour',
       'well differentiated liposarcoma (fnclcc grade 1)',
       'undifferentiated pleomorphic sarcoma.',
 

### Mortality
*TODO - what to do about living and nans*

In [50]:
std_clean(df, "mortality")
idxs = ~df["mortality"].isna()
df.loc[idxs, "mortality"] = df.loc[idxs, "mortality"] \
    .apply(lambda x: -1 if x == "alive" else int(x.split()[1]))

df["mortality"].unique()

array([-1, 39, 85, 56, 83, 68, nan, 76, 82, 52, 61, 72, 77, 44, 87, 79,
       50, 101, 62, 55, 92, 58, 63, 65, 80, 35, 78, 40, 64, 93, 70, 74,
       69, 86, 34, 81, 96, 66, 57, 84, 89, 54, 59, 75, 51, 94, 67, 71, 73,
       90, 95, 17, 47, 88], dtype=object)

### Cancer Mortality
*TODO - split into yes/no and reason*

In [51]:
std_clean(df, "cancer_mortality")
df["cancer_mortality"] = df["cancer_mortality"].str[:3]
df.loc[df["cancer_mortality"] == "?", "cancer_mortality"] = np.nan
df.loc[df["cancer_mortality"] == "yes", "cancer_mortality"] = True
df.loc[df["cancer_mortality"] == "no", "cancer_mortality"] = False

df["cancer_mortality"].unique()

array([nan, False, True], dtype=object)

In [52]:
# TODO
# comment TODO - remove
# query
# misc

## Viewing the cleaned data

In [53]:
df.to_csv("data/clean.csv", index=False)
df

,nhs_num,lab_num,age,gender,location,location_other,biopsy_type,size,mdm2,karyo_code,diagnosis,diagnosis_other,follow_up,mortality,cancer_mortality,cancer_mortality_other,comment,query,misc
0,7079315611,LH24-19172,NaN,NaN,NaN,NaN,NaN,NaN,False,mdm2 normal,NaN,NaN,NaN,-1,NaN,NaN,CF,NaN,NaN
1,4509208588,LH24-20991,74,f,o,breast,NaN,20,False,mdm2 normal,lipoma,NaN,NaN,-1,NaN,NaN,CF,NaN,NaN
2,6342778353,LH24-18651,45,f,hn,right cheek parotid gland,NaN,71,False,mdm2 normal,lipoma,NaN,NaN,-1,NaN,NaN,CF,Should parotid gland be in other?,NaN
3,6415674862,LH24-20355,23,f,lls,NaN,core,NaN,False,mdm2 normal,other (free text),myxoid liposarcoma,NaN,-1,NaN,NaN,CF,Unsure whether diagnosis fits in option 'Pleom...,NaN
4,4520257115,LH24-19486,73,f,uls,NaN,NaN,15,False,mdm2 normal,lipoma,NaN,NaN,-1,NaN,NaN,CF,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2701,6049456305,BH09-3046,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2702,4064416066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 amp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2703,4343743969,LH09-10296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2704,4382331137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mdm2 normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
